# This is my first attempt at constructing a single model to conduct predictictions on stock market technicals. There is probably a more mature version on dev2

In [ ]:
import sys, os
preproc_loc = '/content/drive/MyDrive/WINTER2022/Data40123/TBot/'
if preproc_loc not in sys.path:
  sys.path.insert(0, preproc_loc)
  print(sys.path)

['/content/drive/MyDrive/WINTER2022/Data40123/TBot/', '', '/content', '/env/python', '/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '/usr/local/lib/python3.7/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.7/dist-packages/IPython/extensions', '/root/.ipython']


In [ ]:
## Trains One LSTM model on all Companies in SNP500
import tensorflow as tf
from W_Preproc import Weekly_Preprocessor as WP
import numpy as np
import pandas as pd


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

WP_dict = {
    'train': WP(40, 1970, 2000),
    'val': WP(40, 2001, 2009),
    'test': WP(40, 2010, 2021),
}

# Creates a Generator that will be used by tensorflow to get data
# Takes as input a WeeklyPreprocessor Object
def gen_data(wp_num, batch_size):
    global WP_dict
    wp_name = {0: 'train', 1: 'val', 2: 'test'}[wp_num]
    wp = WP_dict[wp_name]
    while True:
        n_examples = 0
        xs = []
        ys = []
        while n_examples < batch_size:
          result = wp.get_next_week()
          if result is not None:
              x, y, prices, companies, b_date, s_date, cur_week = result
              xs.append(x)
              ys.append(y[:, None])
          else:
              x, y, prices, companies, b_date, s_date, cur_week = wp.get_next_week()
              xs.append(x)
              ys.append(y[:, None])
          n_examples += 1
        yield np.concatenate(xs, axis=0), np.concatenate(ys, axis=0)[:, 0]
        

In [12]:
data_generator = gen_data(0, 25)
val_generator = gen_data(1, 2)
cur_x, cur_y = data_generator.__next__()
val_x, val_y = val_generator.__next__()

model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=cur_x.shape[1:]))
model.add(tf.keras.layers.LSTM(30, return_sequences=True, kernel_regularizer=tf.keras.regularizers.l2(1e-2)))
model.add(tf.keras.layers.LSTM(10, kernel_regularizer=tf.keras.regularizers.l2(1e-2)))
model.add(tf.keras.layers.Dense(10, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-2)))
model.add(tf.keras.layers.Dense(10, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-2)))
model.add(tf.keras.layers.Dense(1, kernel_regularizer=tf.keras.regularizers.l2(1e-2)))

model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss=tf.keras.losses.MeanSquaredError())

for i in range(50):
    model.fit(cur_x, cur_y, epochs=10, batch_size=20, validation_data=(val_x, val_y))
    del cur_x
    del cur_y
    del val_x
    del val_y
    cur_x, cur_y = data_generator.__next__()
    val_x, val_y = val_generator.__next__()

Epoch 1/10
47/47 [==============================] - 11s 118ms/step - loss: 2.2414 - val_loss: 2.0389
Epoch 2/10
47/47 [==============================] - 4s 95ms/step - loss: 1.9278 - val_loss: 1.7711
Epoch 3/10
47/47 [==============================] - 4s 95ms/step - loss: 1.6855 - val_loss: 1.5992
Epoch 4/10
47/47 [==============================] - 4s 95ms/step - loss: 1.5257 - val_loss: 1.4772
Epoch 5/10
47/47 [==============================] - 4s 94ms/step - loss: 1.4109 - val_loss: 1.3780
Epoch 6/10
47/47 [==============================] - 4s 95ms/step - loss: 1.3156 - val_loss: 1.2910
Epoch 7/10
47/47 [==============================] - 4s 95ms/step - loss: 1.2311 - val_loss: 1.2113
Epoch 8/10
47/47 [==============================] - 4s 95ms/step - loss: 1.1545 - val_loss: 1.1381
Epoch 9/10
47/47 [==============================] - 4s 94ms/step - loss: 1.0842 - val_loss: 1.0695
Epoch 10/10
47/47 [==============================] - 4s 94ms/step - loss: 1.0192 - val_loss: 1.0063
Epoch 1

KeyboardInterrupt: ignored

In [20]:
config = model.to_json()
with open('/content/drive/MyDrive/WINTER2022/Data40123/TBot/config.json', 'w') as fout:
  fout.write(config)

weights = model.get_weights()
for i in range(len(weights)):
  np.savetxt('/content/drive/MyDrive/WINTER2022/Data40123/TBot/weights_' + str(i) + '.txt', weights[i])


In [21]:
weights = []
for i in range(len(weights)):
  weights.append(np.loadtxt('/content/drive/MyDrive/WINTER2022/Data40123/TBot/weights_' + str(i) + '.txt'))

In [23]:
with open('/content/drive/MyDrive/WINTER2022/Data40123/TBot/config.json', 'r') as fin:
  config = fin.read()


In [24]:
nm = tf.ker

'{"class_name": "Sequential", "config": {"name": "sequential_6", "layers": [{"class_name": "InputLayer", "config": {"batch_input_shape": [null, 200, 130], "dtype": "float32", "sparse": false, "ragged": false, "name": "input_7"}}, {"class_name": "LSTM", "config": {"name": "lstm_11", "trainable": true, "dtype": "float32", "return_sequences": true, "return_state": false, "go_backwards": false, "stateful": false, "unroll": false, "time_major": false, "units": 30, "activation": "tanh", "recurrent_activation": "sigmoid", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null}}, "recurrent_initializer": {"class_name": "Orthogonal", "config": {"gain": 1.0, "seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "unit_forget_bias": true, "kernel_regularizer": {"class_name": "L2", "config": {"l2": 0.009999999776482582}}, "recurrent_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "re

In [ ]:
test = WP(40, 2001, 2009)
xs = []
ys = []
cur_week = 0
result = test.get_next_week()
while result is not None:
    print(cur_week, end=' ')
    x, y, price, companies, b_date, s_date, cur_week = result
    xs.append(x)
    ys.append(y)
    result = test.get_next_week()
    if cur_week > 10:
        break


In [ ]:
c_ys = []
for y in ys:
    c_ys.append(y[:, None])

val_x = np.concatenate(xs, axis=0)
val_y = np.concatenate(c_ys, axis=0)[:, 0]

In [ ]:
clean_x = val_x[~(np.isnan(val_x).any(axis=2).any(axis=1))]
clean_y = val_y[~(np.isnan(val_x).any(axis=2).any(axis=1))]

In [ ]:
pred = model.predict(clean_x)

In [ ]:
mse = ((clean_y - pred) ** 2).mean()
mse

In [ ]:
(((clean_y > 1) & (pred > 1)) | ((clean_y < 1) & (pred < 1))).mean()

In [ ]:
df = pd.DataFrame({'a':[1, 2, 3], 'b':[4, 5, 6]})

In [ ]:
pd.isna(df).any(axis=1)